In [1]:
import sys, os, math
sys.path.append('..')

import numpy as np
import cv2

from puzzlematch.processing import Draw, Processor
from puzzlematch.hardware import Webcam
from puzzlematch.env import Path
from puzzlematch.ui import display
from puzzlematch import utils

# Edges

In [ ]:
cx,cy,s = 320,240,300
x,y,w,h = cx-s//2, cy-s//2, s, s

camera = Webcam()
camera.open()

while True:
    # Read new frame
    frame = camera.read(hflip=True, vflip=True)
    
    # Crop region of interest
    roi = frame[y:y+h,x:x+w]
    
    # Compute canny edges on each channel
    r_edges = Processor.canny_edges(roi, 'r')
    g_edges = Processor.canny_edges(roi, 'g')
    b_edges = Processor.canny_edges(roi, 'b')
    edges = cv2.add(cv2.add(r_edges,g_edges), b_edges)
    
    # Piece detection based on mask
    box = utils.mask_bbox(edges)
    if box is not None:
        xx,yy,ww,hh = box
        xx,yy = xx+x, yy+y
        frame = Draw.rectangle(frame, p1=(xx,yy), dims=(ww,hh), color=(0,0,255))
    
    # Draw
    frame = Draw.crosshair(frame, center=(cx,cy), size=s)
    r_edges = Draw.text(r_edges, 'Edges on red channel', 'top-left', size=0.4)
    g_edges = Draw.text(g_edges, 'Edges on green channel', 'top-left', size=0.4)
    b_edges = Draw.text(b_edges, 'Edges on blue channel', 'top-left', size=0.4)
    edges   = Draw.text(edges, 'Edges combined', 'top-left', size=0.4)
    
    # Display
    images = [frame, r_edges, g_edges, b_edges, edges]
    key = display(images, wait=1)
    
    if key==27: break
        
camera.close()
cv2.destroyAllWindows()

# Clusters

In [ ]:
def x(y): pass
cv2.namedWindow('window2')
cv2.createTrackbar('k', 'window2', 2, 10, x)

cx,cy,s = 320,240,300
x,y,w,h = cx-s//2, cy-s//2, s, s

camera = Webcam()
camera.open()

while True:
    # Read new frame
    frame = camera.read(hflip=True, vflip=True)
    
    # Crop region of interest
    roi = frame[y:y+h,x:x+w]
    hsv = Processor.hsv(roi)
    bilateral = cv2.bilateralFilter(hsv, 15, 75, 75)
    b,g,r = cv2.split(bilateral)
    threshb, threshg, threshr = Processor.binarize(b), Processor.binarize(g), Processor.binarize(r)
    
    # Compute kmeans clusters
    k = int(cv2.getTrackbarPos('k', 'window2'))
    clusters = Processor.kmeans(bilateral, k)

    # Draw
    frame = Draw.crosshair(frame, center=(cx,cy), size=s)
    bilateral = Draw.text(bilateral, 'Bilateral filter', 'top-left', size=0.4)
    clusters  = Draw.text(clusters, 'KMEANS', 'top-left', size=0.4)
    
    # Display
    images = [frame, bilateral, clusters, b,g,r,threshb, threshg, threshr]
    key = display(images, wait=1)
    if key==27: break
        
camera.close()
cv2.destroyAllWindows()

# HSV

In [2]:
camera = Webcam()
camera.open()

In [ ]:
# def x(y): pass
# cv2.namedWindow('window2')
# cv2.createTrackbar('k', 'window2', 2, 10, x)

cx,cy,s = 320,240,300
x,y,w,h = cx-s//2, cy-s//2, s, s

angle_history = []
record = False
extracted = False
final_angle= None
cropped = None

while True:
    # Read new frame
    frame = camera.read(hflip=True, vflip=True)
    
    # Pipeline
    roi = frame[y:y+h,x:x+w].copy()
    hsv = Processor.hsv(roi)
    hsv = cv2.bilateralFilter(hsv, 15, 75, 75)
    hue,sat,val = cv2.split(hsv)
    mask = Processor.binarize(hue)
    mask = cv2.erode(mask, np.ones((3,3),np.uint8), iterations=1)
    masked = cv2.bitwise_or(roi, roi, mask=mask)
    
    # Orientation
    contours, hierarchy = cv2.findContours(mask, 1, 2)
    contours = contours[0]
    xx,yy,ww,hh = cv2.boundingRect(contours)
    rect = cv2.minAreaRect(contours)
    box = cv2.boxPoints(rect).astype(np.int0)
    
    if extracted:
        rotated_mask = Processor.rotate(mask, final_angle)
        rotated_masked = Processor.rotate(masked, final_angle)
        xxx,yyy,www,hhh = utils.mask_bbox(rotated_mask)
        crop = rotated_masked[yyy:yyy+hhh,xxx:xxx+www]
        if www>hhh: pad1, pad2 = (www-hhh)//2, 0
        else: pad1, pad2 = 0, (hhh-www)//2
            
        rot1 = cv2.copyMakeBorder(crop, pad1, pad1, pad2, pad2, cv2.BORDER_CONSTANT, value=(0,0,0))
        rot2 = Processor.rotate(rot1, 90)
        rot3 = Processor.rotate(rot2, 90)
        rot4 = Processor.rotate(rot3, 90)
        cropped = [rot1,rot2,rot3,rot4]
        extracted = False
    
    if record:
        angle_history.append(rect[-1])
        if len(angle_history)==30:
            final_angle = np.mean(angle_history)
            extracted = True
            record = False
            angle_history = []

    # Draw
    frame = Draw.crosshair(frame, center=(cx,cy), size=s, color=(0,255,0), thickness=1)
    frame = Draw.text(frame, 'Camera input', 'top-left', size=0.4)
    roi = Draw.text(roi, 'ROI', 'top-left', size=0.4)
    hsv = Draw.text(hsv, 'HSV', 'top-left', size=0.4)
    hue = Draw.text(hue, 'Hue', 'top-left', size=0.4)
    mask = Draw.text(mask, 'Mask', 'top-left', size=0.4)
    masked = Draw.text(masked, 'Masked ROI', 'top-left', size=0.4)
    masked = cv2.rectangle(masked, (xx,yy), (xx+ww,yy+hh), (0,255,0))
    masked = cv2.drawContours(masked, [box], 0, (0,0,255))
    
    # Display
    images = [frame, roi, hsv, hue, mask, masked]
    images = (images+cropped if cropped is not None else images)
    key = display(images, wait=1)
    if key==27: break
    if key==ord('a'):
        record = True
        extracted = False
        angle_history = []
        cropped = None
        
# camera.close()
cv2.destroyAllWindows()

In [ ]:
boxhistory = [box,box*0.98,box*0.95,box*1.05]
boxmean = boxhistory[0]
for i in range(1,len(boxhistory)):
    boxmean = boxmean + boxhistory[i]
    
final_box = np.round(boxmean/len(boxhistory)).astype(int)